# Assigning `super_book_codes` to the banned books

**Author:** Michael Falk

**Date:** 31/10/18-1/11/18, 6/11/18, 12/11/18

## Background

One of the key datasets for *Mapping Print, Charting Enlightenment* is a set of documents concerning illegal books in eighteenth-century France. BNF MS 21928-9 contains a list of banned books. It is unclear who exactly wrote the list, but it appears to have been prepared by the central government to assist book inspectors with their tasks across France. BNF Arsenal MS 10305 is an inventory of all the books that were found in the Bastille when it was stormed during the French Revolution. The actual MS has disappeared, but luckily a modern edition exists.

A problem occured during entry of the data. The interface was supposed to oblige the user to assign a 'super book code' to each title in the banned books lists upon entry. But due to a glitch in the interface, the lookup took too long at it was impossible to efficiently do so. Accordingly, only 97 of the 1000+ illegal books have a 'super book code' assigned to them. The data is therefore not linked to the rest of the database and is useless for analysis.

To speed up the process of linking all the data, this notebook uses 'dedupe', an open-source record linkage library, to try and find links between these banned titles and the titles already recorded elsewhere in the database. Hopefully this will speed up record linkage, and provide a testbed for other record linkage tasks in the project.

**Update:** Better versions of the helper functions defined in this notebook have been saved in the file *dedupe_helper_functions.py* in this repo.

***

## Section 1: Import data, initialise model

In [1]:
# Cell 1.1: Load necessary libraries and define key paths
import dedupe as dd
import pandas as pd
import os as os
import time
import numpy as np
import random
from dedupe_helper_functions import dedupe_initialise, run_deduper, save_clusters
import json

input_file = "combined_editions_illegal_books.csv"
output_file = "illegal_books_deduped.csv"
settings_file = "illegal_books_learned_settings"
training_file = "illegal_books_training.json"
output_file = "illegal_books_clustered.csv"
marked_pairs_file = "marked_pairs.json"

Data was preprocessed in R. The full set of 'editions' was extracted from the database. The illegal_titles data was cleaned, and the two datasets were combined into a single large table. The script is in this repo. This preprocessing means that the problem is now a problem of finding duplicate rows in a single table.

In [2]:
# Cell 1.2: Import data
data_frame = pd.read_csv(input_file)

print(f"The data has {data_frame.shape[0]} rows, {data_frame[data_frame['super_book_code'].isna()].shape[0]} of which need super_book_codes assigned.")

The data has 17164 rows, 1921 of which need super_book_codes assigned.


In [ ]:
# Cell 1.3: Initialise deduper

# Creat a list of fields for the model to look at. NB: 'ID' and 'UUID' are not relevant to the task,
# hence do not appear in the list. The 'super_book_code' also encodes no useful information,
# because the problem is that we have records without codes, and the model will learn to focus on that
# column too much if we include it, since it is a nearly perfect determinant of identity.
fields = [
    {'field':'full_book_title', 'type': 'String'},
    {'field':'author_name', 'type': 'String'},
    {'field':'stated_publication_places', 'type': 'String'},
    {'field':'stated_publication_years', 'type': 'DateTime'}
]

# This line creates the Dedupe model. If it finds a file at the 'settings_file', it will load as a 'static'
# deduper that can be used efficiently to cluster data, but that cannot be trained. If there is no
# settings file at the given path, it will initialise as an active deduper that must be trained before it can be used.
deduper = dedupe_initialise(data_frame, fields, settings_file, training_file)

## Section 2: Training the Model

In this section of the notebook, we give training data to the model so it can learn to identify which books are the same.

On 12/12/18, MF tried an alternative training regimen, where the model only looked at the title of the book. It did not do well! Only 9 clusters were found.

In [ ]:
# Cell 2.1: Adding training data to the model (console)

# Run this cell to open the console labeller, which allows you to manually enter training data in the output window.
dd.consoleLabel(deduper)

In [5]:
# Cell 2.2: Adding training data to the model (marked pairs json file)

# Run this cell to import the training json file generated by 'illegal_books_get_marked_pairs.R', and add it to the model.
with open(marked_pairs_file, 'r') as f:
    marked_pairs = json.load(f)

In [7]:
# Cell 2.3: Add the marked pairs to model
deduper.markPairs(marked_pairs)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (metaphoneToken, full_book_title)
INFO:dedupe.training:SimplePredicate: (commonFourGram, full_book_title)


## Section 3: Inspecting the results

In [ ]:
# Cell 3.1. Add cluster data back to original data frame and save to csv
_ = save_clusters(matches, data_frame, output_file)

In [ ]:
# Cell 3.2. Sanity check. How good are the model's assignments?
# Run this cell a few times to look at different random clusters
data_frame[data_frame['cluster'] == random.randint(0, len(matches) + 1)]

In [18]:
# Cell 3.3. How much time have we saved? How many illegal books have been assigned a super_book_code?
assigned = data_frame[
    pd.notnull(data_frame['cluster']) & # which books have been assigned a cluster?
    pd.notnull(data_frame['UUID']) & # only illegal books have UUIDs
    pd.isna(data_frame['super_book_code']) # only interested in books that didn't already have super_book_codes
].shape[0]

total = data_frame[
    pd.notnull(data_frame['UUID']) &
    pd.isna(data_frame['super_book_code'])
].shape[0]

print(f"{assigned} illegal books have been given super_book_codes, of {total} that lack them.")

532 illegal books have been given super_book_codes, of 1921 that lack them.


We can consider the accuracy of the model more accurately by seeing how often it clustered books with different super_book_codes.

In [27]:
# Cell 3.4. Inspecting the super_book_codes in all the clusters.
multi_groups = (data_frame.groupby(by="cluster")['super_book_code'] # group into clusters, inspect 'super_book_code'
               .nunique() # count how many unqiue 'super_book_codes' are in the cluster
               .where(lambda x: x > 1) # only keep clusters with more than one 'super_book_code'
               .dropna()) # drop the NaNs created by .where()

print(f"Of the {data_frame.cluster.max()} clusters found by dedupe, {len(multi_groups)} contain multiple superbooks.")

Of the 3386.0 clusters found by dedupe, 717 contain multiple superbooks.


In [ ]:
# Cell 3.5. Which books has Dedupe confounded?
# Pick one of the groups
rand_multi = int(random.choice(multi_groups.index.tolist()))

# Inspect it
data_frame[data_frame['cluster'] == rand_multi]

In [ ]:
# Are there any clusters where it has put more than one illegal book?
multi_illegal = (data_frame[pd.notnull(data_frame['UUID'])]
                 .groupby(by = 'cluster')['UUID']
                 .nunique()
                 .where(lambda x: x > 1)
                 .dropna())

print(f"Of the {len(matches)} clusters found by dedupe, {len(multi_illegal)} contain multiple illegal books.")

In [ ]:
# Examine some of these ones:
# Pick one of the groups
rand_multi = int(random.choice(multi_illegal.index.tolist()))

# Inspect it
data_frame[data_frame['cluster'] == rand_multi]

## Section 4: Using the results

The question is: how to use the results? The most obvious course seems to be to be this: go through all the illegal books, and keep the most confident result that the model has put out. Then we can manually go over them, and if they are okay, update the database.

In [9]:
# Run this cell if you are revisiting this notebook after training, and need to load
# the clusters from a previous run.
data_frame = pd.read_csv(output_file)

In [15]:
matched_illegals = []
top_cluster = int(data_frame['cluster'].max())

for i in range(top_cluster): # loop over all clusters
    this_slice = data_frame[data_frame.cluster == i] # inspect this cluster
    if any(pd.notnull(this_slice.UUID) & # if there is a banned book ...
        pd.isnull(this_slice.super_book_code) # ... without a super_book_code...
    ) & any(pd.isnull(this_slice.UUID)): # ... and there a super_book_code in the cluster, then:
        idx = this_slice[pd.notnull(this_slice.super_book_code)].confidence.idxmax() # find the highest confidence row with an sbc
        sbc = this_slice.loc[idx].super_book_code # get the super book code
        
        banned_books = this_slice[pd.notnull(this_slice.UUID)] # get the banned books in the slice
        
        for book in banned_books.itertuples(): # loop over banned books
            new_match = {}
            new_match['UUID'] = book.UUID
            new_match['super_book_code'] = sbc
            new_match['confidence'] = book.confidence
            matched_illegals.append(new_match)

In [21]:
str(matched_illegals[22]['confidence'])

'0.7842538356781006'

The below cell writes the results into an SQL file that can be used to update the illegal books table.

In [25]:
# Write this out as an SQL command
sql_statement = "USE manuscripts;\nALTER TABLE manuscript_titles_illegal ADD confidence FLOAT;\n"

for match in matched_illegals:
    sql_statement += 'UPDATE manuscript_titles_illegal\n'
    sql_statement += 'SET illegal_super_book_code = "' + match['super_book_code'] + '",\n'
    sql_statement += 'confidence = "' + str(match['confidence']) + '"\n'
    sql_statement += 'WHERE UUID = "' + match['UUID'] + '";\n'
    
with open('update_illegal_books.sql', 'w', encoding='utf-8') as sql_file:
    sql_file.write(sql_statement)

## Conclusion

Dedupe appears to do a good job in linking the illegal books to super books that are already in the dataset. The question is whether the ~1600 books the algorithm could not cluster are new books that aren't already in our dataset, or if the algorithm has low recall.

Manual investigation will be necessary to see if the ~1600 super books are already in the database.

It may also be possible to tune the model further, by
1. giving it more training data, or
2. increasing the `recall_weight` so that the model cares more about finding possible matches than being accurate when it does find them.

**Addendum (12/11/2018):** Simon reckons that finding `super_book_codes` for ~300 of the illegal books is unsurprising. His hunch is that the authorities were quite good at extinguishing banned titles most of the time in *ancien r&eacute;gime* France.

**Addendum (19/11/2018):** I tried generating a whole lot of training data, using the super book codes to find matching and non-matching pairs. Feeding this to the model using the `Dedupe.markPairs()` method, the model's recall jumped considerably, and it has found 600+ matches with the illegal books. Now to check that data and upload it...

**Addendum (12/12/2018):** Chat with SB on 11/12. We investigated how many books are likely to already be in the dataset. Only 183 of the Bastille Register books were in FBTEE-1, so finding 400 clusters seems a reasonable amount.